In [198]:
import os
from glob import glob
import pandas as pd
import numpy as np

In [199]:
root = 'data'
pathCsv = os.path.join(root, 'HAM10000_metadata.csv')
df = pd.read_csv(pathCsv)

In [200]:
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [201]:
df.isnull().sum()

lesion_id        0
image_id         0
dx               0
dx_type          0
age             57
sex              0
localization     0
dtype: int64

In [202]:
df['age'].fillna((df['age'].mean()), inplace=True)
df.isnull().sum()

lesion_id       0
image_id        0
dx              0
dx_type         0
age             0
sex             0
localization    0
dtype: int64

In [203]:
df['Class'] = pd.Categorical(df['dx']).codes

Path = {}
for item in glob(os.path.join(root, 'reshaped', '*.jpg')):
    filename = os.path.splitext(os.path.basename(item))[0]
    Path[filename] = item

df['path'] = df['image_id'].map(Path.get)

In [204]:
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,Class,path
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,2,data/reshaped/ISIC_0027419.jpg
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,2,data/reshaped/ISIC_0025030.jpg
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,2,data/reshaped/ISIC_0026769.jpg
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,2,data/reshaped/ISIC_0025661.jpg
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,2,data/reshaped/ISIC_0031633.jpg


In [205]:
from PIL import Image
df['image'] = df['path'].map(lambda x: np.asarray(Image.open(x)))
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,Class,path,image
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,2,data/reshaped/ISIC_0027419.jpg,"[[[190, 155, 188], [190, 155, 188], [190, 154,..."
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,2,data/reshaped/ISIC_0025030.jpg,"[[[22, 15, 33], [20, 11, 30], [25, 13, 33], [3..."
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,2,data/reshaped/ISIC_0026769.jpg,"[[[175, 127, 141], [181, 131, 143], [192, 138,..."
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,2,data/reshaped/ISIC_0025661.jpg,"[[[29, 9, 18], [28, 9, 15], [44, 21, 27], [56,..."
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,2,data/reshaped/ISIC_0031633.jpg,"[[[133, 87, 113], [151, 103, 129], [163, 115, ..."


In [206]:
df['dx'].value_counts()

nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: dx, dtype: int64

In [207]:
df_nv = df[df.dx=='nv']
df_mel = df[df.dx=='mel']
df_bkl = df[df.dx=='bkl']
df_bcc = df[df.dx=='bcc']
df_akiec = df[df.dx=='akiec']
df_vasc = df[df.dx=='vasc']
df_df = df[df.dx=='df']
df_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,Class,path,image
1095,HAM_0005276,ISIC_0027008,df,histo,75.0,male,back,3,data/reshaped/ISIC_0027008.jpg,"[[[155, 110, 105], [166, 121, 116], [178, 133,..."
1096,HAM_0005276,ISIC_0028790,df,histo,75.0,male,back,3,data/reshaped/ISIC_0028790.jpg,"[[[205, 159, 169], [212, 167, 174], [217, 170,..."
1097,HAM_0004103,ISIC_0028880,df,histo,55.0,male,lower extremity,3,data/reshaped/ISIC_0028880.jpg,"[[[221, 187, 212], [223, 189, 214], [223, 190,..."
1098,HAM_0004103,ISIC_0031309,df,histo,55.0,male,lower extremity,3,data/reshaped/ISIC_0031309.jpg,"[[[30, 14, 25], [46, 28, 40], [63, 45, 57], [8..."
1099,HAM_0001164,ISIC_0030021,df,histo,50.0,female,upper extremity,3,data/reshaped/ISIC_0030021.jpg,"[[[198, 155, 162], [198, 155, 162], [198, 155,..."


In [208]:
from sklearn.utils import resample

df_df_upsample = resample(df_df, 
                                 replace=True,     
                                 n_samples=1000,    
                                 random_state=123)
df_nv_downsample = resample(df_nv, 
                                 replace=True,     
                                 n_samples=1000,    
                                 random_state=123)
df_mel_upsample = resample(df_mel, 
                                 replace=True,     
                                 n_samples=1000,    
                                 random_state=123)
df_bkl_upsample = resample(df_bkl, 
                                 replace=True,     
                                 n_samples=1000,    
                                 random_state=123)
df_bcc_upsample = resample(df_bcc, 
                                 replace=True,     
                                 n_samples=1000,    
                                 random_state=123)
df_akiec_upsample = resample(df_akiec, 
                                 replace=True,     
                                 n_samples=1000,    
                                 random_state=123)
df_vasc_upsample = resample(df_vasc, 
                                 replace=True,     
                                 n_samples=1000,    
                                 random_state=123)



In [209]:
df.columns

Index(['lesion_id', 'image_id', 'dx', 'dx_type', 'age', 'sex', 'localization',
       'Class', 'path', 'image'],
      dtype='object')

In [210]:
df_resampled = pd.DataFrame(columns=['lesion_id', 'image_id', 'dx', 'dx_type', 'age', 'sex', 'localization',
        'path', 'image'],
      dtype='object')

In [211]:
df_resampled.shape

(0, 9)

In [212]:
df_resampled = pd.concat([df_resampled, df_df_upsample])
df_resampled.shape

/Users/shalu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


(1000, 10)

In [213]:
df_resampled = pd.concat([df_resampled, df_nv_downsample])
df_resampled.shape

/Users/shalu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


(2000, 10)

In [214]:
df_resampled = pd.concat([df_resampled, df_mel_upsample])
df_resampled.shape

/Users/shalu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


(3000, 10)

In [215]:
df_resampled = pd.concat([df_resampled, df_bkl_upsample])
df_resampled.shape

/Users/shalu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


(4000, 10)

In [216]:
df_resampled = pd.concat([df_resampled, df_bcc_upsample])
df_resampled.shape

/Users/shalu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


(5000, 10)

In [217]:
df_resampled = pd.concat([df_resampled, df_akiec_upsample])
df_resampled.shape

/Users/shalu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


(6000, 10)

In [218]:
df_resampled = pd.concat([df_resampled, df_vasc_upsample])
df_resampled.shape

/Users/shalu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


(7000, 10)

In [219]:
df = df_resampled
df['dx'].value_counts()

vasc     1000
bcc      1000
mel      1000
df       1000
akiec    1000
bkl      1000
nv       1000
Name: dx, dtype: int64

In [220]:
df.shape

(7000, 10)

In [221]:
df.head()

,Class,age,dx,dx_type,image,image_id,lesion_id,localization,path,sex
1204,3.0,35.0,df,consensus,"[[[235, 175, 187], [231, 171, 183], [231, 171,...",ISIC_0025314,HAM_0000757,lower extremity,data/reshaped/ISIC_0025314.jpg,female
1161,3.0,40.0,df,consensus,"[[[185, 159, 198], [182, 156, 195], [182, 156,...",ISIC_0024330,HAM_0006188,lower extremity,data/reshaped/ISIC_0024330.jpg,male
1187,3.0,60.0,df,consensus,"[[[189, 133, 136], [190, 134, 137], [192, 135,...",ISIC_0026471,HAM_0006787,lower extremity,data/reshaped/ISIC_0026471.jpg,male
1193,3.0,55.0,df,consensus,"[[[228, 157, 163], [227, 156, 162], [228, 157,...",ISIC_0030665,HAM_0005359,lower extremity,data/reshaped/ISIC_0030665.jpg,female
1197,3.0,65.0,df,consensus,"[[[220, 147, 164], [220, 147, 164], [221, 148,...",ISIC_0026629,HAM_0004065,lower extremity,data/reshaped/ISIC_0026629.jpg,female


In [222]:
df.to_csv('saved.csv')

In [223]:
temp = df['image'][1].shape
count = 0
for i in range(df.shape[0]):
    count += 1
print("Dimensions of image: {}, number: {}".format(temp, count))
    

Dimensions of image: (75, 100, 3), number: 7000


In [224]:
from sklearn.model_selection import train_test_split
x=df.drop(['Class'],axis=1)
y=df['Class']
xTrain, xTest, yTrain, yTest  = train_test_split(x, y, test_size=0.20, random_state=42)

In [225]:
x_train = np.asarray(xTrain['image'].tolist())
x_test = np.asarray(xTest['image'].tolist())
xTrainmean = np.mean(x_train)
xTrainstd = np.std(x_train)
xTestmean = np.mean(x_test)
xTeststd = np.std(x_test)
x_train = (x_train - xTrainmean)/xTrainstd
x_test = (x_test - xTestmean)/xTeststd

In [226]:
import keras
from keras.utils.np_utils import to_categorical

y_train = to_categorical(yTrain, num_classes = 7)
y_test = to_categorical(yTest, num_classes = 7)

In [227]:
np.save("test_data_x", x_test)
np.save("test_data_y", yTest)

In [228]:
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = 0.1, random_state = 2)
x_train = x_train.reshape(x_train.shape[0], *(75, 100, 3))
x_test = x_test.reshape(x_test.shape[0], *(75, 100, 3))

In [229]:
x_validate = x_validate.reshape(x_validate.shape[0], *(75, 100, 3))

In [230]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
import itertools
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

dimensions = (75, 100, 3)
Classes = 7

model = Sequential()

model.add(Conv2D(32, (3, 3),activation='relu',padding = 'Same',input_shape=dimensions))
model.add(Conv2D(32, (3, 3), activation='relu',padding = 'Same',))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(Classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 75, 100, 32)       896       
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 75, 100, 32)       9248      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 37, 50, 32)        0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 37, 50, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 37, 50, 64)        18496     
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 37, 50, 64)        36928     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 18, 25, 64)        0         
__________

In [231]:
Info = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False, 
        featurewise_std_normalization=False, 
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=10, 
        zoom_range = 0.1,
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=False,
        vertical_flip=False) 

Info.fit(x_train)

In [ ]:
Adam_Optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#SGD_Optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
model.compile(optimizer = Adam_Optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
Lap = 50 
Volume = 10
history = model.fit_generator(Info.flow(x_train,y_train, batch_size=Volume),
                              epochs = Lap, validation_data = (x_validate,y_validate),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // Volume
                             )

Epoch 1/50
504/504 [==============================] - 175s 348ms/step - loss: 1.7947 - acc: 0.2738 - val_loss: 1.4906 - val_acc: 0.4125
Epoch 2/50
504/504 [==============================] - 170s 338ms/step - loss: 1.5217 - acc: 0.4187 - val_loss: 1.2980 - val_acc: 0.5036
Epoch 3/50
504/504 [==============================] - 170s 337ms/step - loss: 1.3981 - acc: 0.4496 - val_loss: 1.2469 - val_acc: 0.5393
Epoch 4/50
504/504 [==============================] - 167s 330ms/step - loss: 1.3077 - acc: 0.4968 - val_loss: 1.2245 - val_acc: 0.5250
Epoch 5/50
504/504 [==============================] - 168s 333ms/step - loss: 1.2331 - acc: 0.5268 - val_loss: 1.1637 - val_acc: 0.5732
Epoch 6/50
504/504 [==============================] - 167s 331ms/step - loss: 1.1772 - acc: 0.5480 - val_loss: 1.0590 - val_acc: 0.5929
Epoch 7/50
504/504 [==============================] - 1787s 4s/step - loss: 1.1341 - acc: 0.5573 - val_loss: 0.9923 - val_acc: 0.6429
Epoch 8/50
504/504 [==============================

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=1)
loss_v, accuracy_v = model.evaluate(x_validate, y_validate, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))
model.save("model_1000samples_50.h5")

In [ ]:
#1. Function to plot model's validation loss and validation accuracy
import matplotlib.pyplot as plt
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [ ]:
plot_model_history(history)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(x_test[86])
plt.show()

In [ ]:
#Testing Models
from keras.models import load_model


new_model1 = load_model('backups/model_1000samples_20.h5')
prediction1 = new_model1.predict(x_test)
print(np.argmax(prediction1[0]))
loss, accuracy = new_model1.evaluate(x_test, y_test, verbose=1)
loss_v, accuracy_v = new_model1.evaluate(x_validate, y_validate, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

In [ ]:
new_model2 = load_model('backups/model_1500samples_75.h5')
prediction2 = new_model2.predict(x_test)
print(np.argmax(prediction2[0]))
loss, accuracy = new_model2.evaluate(x_test, y_test, verbose=1)
loss_v, accuracy_v = new_model2.evaluate(x_validate, y_validate, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

In [ ]:
new_model3 = load_model('backups/model_1500samples_20.h5')
prediction3 = new_model3.predict(x_test)
print(np.argmax(prediction3[0]))
loss, accuracy = new_model3.evaluate(x_test, y_test, verbose=1)
loss_v, accuracy_v = new_model3.evaluate(x_validate, y_validate, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

In [ ]:
new_model4 = load_model('backups/model_1500samples_20layers_4.h5')
prediction4 = new_model4.predict(x_test)
print(np.argmax(prediction4[0]))
loss, accuracy = new_model4.evaluate(x_test, y_test, verbose=1)
loss_v, accuracy_v = new_model4.evaluate(x_validate, y_validate, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

In [ ]:
new_model5 = load_model('backups/initial_model.h5')
prediction5 = new_model5.predict(x_test)
print(np.argmax(prediction5[0]))
loss, accuracy = new_model5.evaluate(x_test, y_test, verbose=1)
loss_v, accuracy_v = new_model5.evaluate(x_validate, y_validate, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

In [ ]:
new_model6 = load_model('model_1500samples_20_keras_SGD.h5')
prediction6 = new_model6.predict(x_test)
print(np.argmax(prediction6[0]))
loss, accuracy = new_model6.evaluate(x_test, y_test, verbose=1)
loss_v, accuracy_v = new_model6.evaluate(x_validate, y_validate, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

In [ ]:
new_model7 = load_model('model_1500samples_50_keras_SGD.h5')
prediction7 = new_model7.predict(x_test)
print(np.argmax(prediction7[0]))
loss, accuracy = new_model7.evaluate(x_test, y_test, verbose=1)
loss_v, accuracy_v = new_model7.evaluate(x_validate, y_validate, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))

In [ ]:
new_model8 = load_model('backups/initial_model_50.h5')
prediction8 = new_model8.predict(x_test)
print(np.argmax(prediction8[0]))
loss, accuracy = new_model8.evaluate(x_test, y_test, verbose=1)
loss_v, accuracy_v = new_model8.evaluate(x_validate, y_validate, verbose=1)
print("Validation: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))